# FQE

In [1]:
import fqe
import openfermion as of
from openfermion import FermionOperator as fo
import tequila as tq
import numpy as np
import time

In [2]:
geom = """
H 0.0 0.0 0.0
H 0.0 0.0 1.0
"""

# Create Hamiltonian
mol = tq.Molecule(geometry=geom, basis_set="sto-3g", transformation='reorderedjordanwigner')
HF = mol.make_molecular_hamiltonian()
HH = of.transforms.get_fermion_operator(HF)
HH = fqe.get_hamiltonian_from_openfermion(HH, mol.n_orbitals)

# Reference wfn

In [3]:
print(tq.simulate(mol.prepare_reference()))
wfn = fqe.Wavefunction([[mol.n_electrons, 0, mol.n_orbitals]])
# wfn.set_wfn(strategy="hartree-fock")
# wfn.set_wfn(strategy="ones")
# wfn.set_wfn(strategy="random")
# print()
# for term in wfn._civec:
#     print()
#     print(term,'->',vars(wfn._civec[term]))
#     print(term,'->',wfn._civec[term].coeff, wfn._civec[term].coeff.shape)
#     print()
coeff = np.array([[1+0j,0+0j],[0+0j,0+0j]]) # Hartree-Fock
wfn.set_wfn(strategy='from_data', raw_data={(mol.n_electrons,0): coeff})
wfn.print_wfn()
# energy = wfn.expectationValue(HH)
# print(f"fqe: {energy.real}")
print()

+1.0000 |1010> 
Sector N = 2 : S_z = 0
a'01'b'01' (1+0j)



# Excitation operators

In [4]:
angle = -np.pi
# Double excitation
# print(tq.simulate(mol.prepare_reference()+mol.make_excitation_gate([(0,2),(1,3)],angle)))
# op = mol.make_excitation_generator([(0,2),(1,3)], fermionic=True) # op = 1j*fo('0^ 2 1^ 3') -1j*fo('3^ 1 2^ 0')
# wfn_t = wfn.time_evolve(0.5*angle, op)
# wfn_t.print_wfn()
# print()

# Single excitation
# print(tq.simulate(mol.prepare_reference()+mol.make_excitation_gate([(0,2)],angle)))
# op = mol.make_excitation_generator([(0,2)], fermionic=True)
# wfn_t = wfn.time_evolve(0.5*angle, op)
# wfn_t.print_wfn()
# print()

print(tq.simulate(mol.prepare_reference()+mol.make_excitation_gate([(1,3)],angle)))
op = mol.make_excitation_generator([(1,3)], fermionic=True)
wfn_t = wfn.time_evolve(0.5*angle, op)
wfn_t.print_wfn()
print()

+1.0000 |1001> 
Sector N = 2 : S_z = 0
a'01'b'10' (1+0j)



# Expectation Value

In [5]:
# U = mol.prepare_reference()+mol.make_excitation_gate([(0,2),(1,3)],angle)
# U = mol.prepare_reference()+mol.make_excitation_gate([(0,2)],angle)
U = mol.prepare_reference()+mol.make_excitation_gate([(1,3)],angle)
print(f"tequila: {tq.simulate(tq.ExpectationValue(U,mol.make_hamiltonian()))}")
energy = wfn_t.expectationValue(HH)
print(f"fqe:     {energy.real}")

tequila: -0.5490812095500966
fqe:     -0.5490812095500969


# Braket

In [ ]:
angle = -np.pi/2

teq_circ1 = mol.prepare_reference()+mol.make_excitation_gate([(0,2),(1,3)],angle)
print(tq.simulate(teq_circ1))
op = mol.make_excitation_generator([(0,2),(1,3)], fermionic=True) # op = 1j*fo('0^ 2 1^ 3') -1j*fo('3^ 1 2^ 0')
wfn1 = wfn.time_evolve(0.5*angle, op)
wfn1.print_wfn()
print()

teq_circ2 = mol.prepare_reference()+mol.make_excitation_gate([(0,2)],angle)
print(tq.simulate(teq_circ2))
op = mol.make_excitation_generator([(0,2)], fermionic=True)
wfn2 = wfn.time_evolve(0.5*angle, op)
wfn2.print_wfn()
print()

# teq_circ3 = mol.prepare_reference()+mol.make_excitation_gate([(1,3)],angle)
# print(tq.simulate(teq_circ3))
# op = mol.make_excitation_generator([(1,3)], fermionic=True)
# wfn3 = wfn.time_evolve(0.5*angle, op)
# wfn3.print_wfn()
# print()

print("tequila:", tq.simulate(tq.BraKet(ket=teq_circ1, bra=teq_circ2, operator=mol.make_hamiltonian())[0]))
print("FQE:", wfn1.expectationValue(HH, brawfn=wfn2).real)

+0.7071 |1010> +0.7071 |0101> 
Sector N = 2 : S_z = 0
a'01'b'01' (0.7071067811865476+0j)
a'10'b'10' (0.7071067811865475+0j)

+0.7071 |1010> +0.7071 |0110> 
Sector N = 2 : S_z = 0
a'01'b'01' (0.7071067811865476+0j)
a'10'b'01' (0.7071067811865475+0j)

tequila: -0.43465903291623054
FQE: -0.4346590329162334
